## 

# Crew AI - Book Search

## Import libraries

In [1]:
import os
from crewai import Crew, Task, Agent

# Warning control
import warnings
warnings.filterwarnings('ignore')

## Configuration

In [2]:
# ------ Ollama - Local---------#
os.environ["LOCAL_EMBEDDINGS"]="http://localhost:11434/api/embeddings"
embeddings_model = "nomic-embed-text:v1.5"
os.environ["LOCAL_EMBEDDINGS_MODEL"]=embeddings_model # documentar como definir variáveis de ambiente

# ------ OpenAI - API ---------#
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# ------ DEBUG ---------#
os.environ["DEBUG"]="False"
debug = os.getenv("DEBUG_MODE", "False")
if debug.lower() == "true":
    VERBOSE = 2
else:
    VERBOSE = False

### Crew - Busca no contexto

In [3]:
from langchain_openai import ChatOpenAI
from search.context import utils

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

from crewai_tools import tool

# Configurações locais
LOCAL_EMBEDDINGS_URL = "http://localhost:11434/api/embeddings"
EMBEDDINGS_MODEL = "nomic-embed-text:v1.5"
os.environ["LOCAL_EMBEDDINGS"] = LOCAL_EMBEDDINGS_URL
os.environ["LOCAL_EMBEDDINGS_MODEL"] = EMBEDDINGS_MODEL

# Configurações da API OpenAI
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Configurações de DEBUG
os.environ["DEBUG"] = "False"
DEBUG_MODE = os.getenv("DEBUG_MODE", "False").lower() == "true"
VERBOSE = 2 if DEBUG_MODE else False

# Constantes Globais
AVATARS = {"commentary": "./assets/e-book.png", "bible": "./assets/biblia.png",
           "revisor": "./assets/revisor.png"}
VECTOR_STORE = 'chroma'
PATH_BOOKS = f"./vector-store/{VECTOR_STORE}/books"
PATH_BIBLE = f"./vector-store/{VECTOR_STORE}/bible"
EMBEDDINGS_MODEL = os.getenv("LOCAL_EMBEDDINGS_MODEL")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
DEBUG = os.getenv("DEBUG_MODE", "False").lower() == "true"
VERBOSE = 2 if DEBUG else False


# ------- TOOLS ------- #

@tool("Pesquisar em Livros")
def library_tool(text: str) -> str:
    """Retorna livros que possuem conteúdo relacionado com termo ou assunto buscado."""
    result = ""
    db = utils.retriever_context(embeddings_model=EMBEDDINGS_MODEL, path_books=PATH_BOOKS, vector_store=VECTOR_STORE)
    documents = db.similarity_search(text)
    for row in documents:
        page_content = row.page_content
        source = row.metadata['source'].replace('./books/', '').replace('.pdf', '')
        page = row.metadata['page']
        result += f"**Trecho de {source} - página {page}:** \n{page_content}\n\n"
    return result


# ------- AGENTS ------- #

library_agent = Agent(
    role="Especialista em Pesquisa em Biblioteca",
    goal="Encontre os trechos de livros que falem explicitamente do tópico fornecido.",
    backstory=(
        "Você atualmente é encarregado de usar a library_tool "
        "para encontrar trechos que falem do tópico fornecido. "
        "Seu objetivo é garantir que os trechos que você fornece "
        "tratem diretamente do tópico e não uma relação distante. "
    ),
    tools=[library_tool],
    allow_delegation=False,
    verbose=VERBOSE,
    llm=llm
)

revisor_agent = Agent(
    role="Revisor de Texto",
    goal="Revisar e melhorar a qualidade dos trechos fornecidos pelos Especialistas.",
    backstory=(
        "Você atualmente é encarregado de revisar e melhorar a qualidade "
        "dos trechos fornecidos pelos Especialistas em Pesquisa da Biblioteca. "
        "Seu objetivo é garantir que os trechos fornecidos são relevantes, "
        "precisos e falem do tópico fornecido pelo cliente."
    ),
    allow_delegation=False,
    verbose=VERBOSE,
    llm=llm,
)

# ------- TASKS ------- #

library_search_task = Task(
    description=(
        "Sua tarefa é usar a library_tool para encontrar trechos de livros que falem explicitamente "
        " do tópico: {text} "
    ),
    expected_output=(
        "Trechos de livros que falem explicitamente do tópico fornecido. "
        "Cada trecho deve ser claramente citado e incluída sua fonte, com o título do livro "
        "autor e número da página."
    ),
    agent=library_agent,
)

library_select_task = Task(
    description=(
        "Selecione os trechos que falam sobre o tópico: {text} "
        "Explique a relação ou indique caso não haja relação. Responda em português."
    ),
    expected_output=(
        "Tópico fornecido pelo cliente."
        "Trechos que falam do tópico fornecido pelo cliente, citando com sua fonte, incluindo o título do livro "
        "autor e número da página ou indicação caso não haja relação. "
        "Explicação onde o trecho fala diretamente sobre o tópico fornecido pelo cliente."
    ),
    agent=revisor_agent,
)

In [4]:
crew = Crew(
  agents=[library_agent, revisor_agent],
  tasks=[library_search_task, library_select_task],
  verbose=VERBOSE,
  memory=False,
)

In [5]:
input_text  = """Julgamento do trono branco"""
input_text

'Julgamento do trono branco'

In [6]:
inputs = {
    "text": input_text
}
result = crew.kickoff(inputs=inputs)

 

**Trecho de Somos Todos Teólogos - RC Sproul - página 210:** 
Quando, porém, vier o Consolador, que eu vos enviarei da parte do Pai, o Espírito
da verdade, que dele procede, esse dará testemunho de mim; e vós também
testemunhareis, porque estais comigo desde o princípio.
Tenho-vos dito estas coisas para que não vos escandalizeis. Eles vos expulsarão
das sinagogas; mas vem a hora em que todo o que vos matar julgará com isso
tributar culto a Deus. Isto farão porque não conhecem o Pai, nem a mim. Ora, estas
coisas vos tenho dito para que, quando a hora chegar, vos recordeis de que eu vo-
las disse. Não vo-las disse desde o princípio, porque eu estava convosco (Jo 15.24-
16.4).
 
OUTRO CONSOLADOR

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 68:** 
não der certo, se irromper uma guerra? E se amanhã amanhecer chovendo, e os ônibus 
estiverem em greve, ou se houver um terremoto ? E se alguém roubar o meu dinheiro, invadir 
minha casa, violentar minha filha ou me matar?"74 

In [7]:
from IPython.display import display, Markdown

def get_task_output(task):
    return f"""
**Tarefa:** \n
{task.output.description} \n \n
**Saída:** \n
{task.output.raw_output}
"""

Markdown(get_task_output(library_search_task))



**Tarefa:** 

O cliente forneceu o seguinte tópico para análise:
Julgamento do trono branco

Sua tarefa é usar a library_tool para encontrar trechos de livros que falem explicitamente deste tópico.  
 

**Saída:** 

**Trecho de Somos Todos Teólogos - RC Sproul - página 210:** 
Quando, porém, vier o Consolador, que eu vos enviarei da parte do Pai, o Espírito
da verdade, que dele procede, esse dará testemunho de mim; e vós também
testemunhareis, porque estais comigo desde o princípio.
Tenho-vos dito estas coisas para que não vos escandalizeis. Eles vos expulsarão
das sinagogas; mas vem a hora em que todo o que vos matar julgará com isso
tributar culto a Deus. Isto farão porque não conhecem o Pai, nem a mim. Ora, estas
coisas vos tenho dito para que, quando a hora chegar, vos recordeis de que eu vo-
las disse. Não vo-las disse desde o princípio, porque eu estava convosco (Jo 15.24-
16.4).
 
OUTRO CONSOLADOR

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 68:** 
não der certo, se irromper uma guerra? E se amanhã amanhecer chovendo, e os ônibus 
estiverem em greve, ou se houver um terremoto ? E se alguém roubar o meu dinheiro, invadir 
minha casa, violentar minha filha ou me matar?"74 
Quando essas perguntas passam a guiar nossa vida, estamos refinanciando nossa hipoteca 
na casa do medo. 
Jesus diz simplesmente: "Permanecei em mim, e eu permanecerei em vós" (Jo 15:4)- Essa 
permanência não é uma mansão celeste na vida futura, mas um lugar seguro bem no meio do 
nosso mundo de ansiedade. "Se alguém me ama, guardará a minha palavra; e meu Pai o amará, 
e viremos para ele e faremos nele morada" (Jo  14:23).

**Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill - página 45:** 
está em nós mesmos? ” Mas eu gostaria de afiar bem o meu bisturi e 
aprofundá -lo um pouco mais nos pregadores:  passou a época dos 
grandes ser mões tipo “lanche rápido ”, temperados com tiradas 
humorísticas para tentar estimular o fraco apetite espiritual do homem 
de nossos dias? Ou estamos nos esforçando para comunicar os 
“poderes do mundo vindouro ” em todos os cultos?  
Pensemos um pouco em Paulo. Após recebe r uma poderosa unção 
do Espírito Santo, ele saiu pela Ásia menor para travar ali uma 
intensa batalha espiritual, causando agitação  nos mercados, 
sinagogas e palácios. E ia a toda parte, tendo no coração e nos

**Trecho de Somos Todos Teólogos - RC Sproul - página 213:** 
Tenho ainda muito que vos dizer, mas vós não o podeis suportar agora; quando
vier, porém, o Espírito da verdade, ele vos guiará a toda a verdade; porque não
falará por si mesmo, mas dirá tudo o que tiver ouvido e vos anunciará as coisas que
hão de vir. Ele me glorificará, porque há de receber do que é meu e vo-lo há de
anunciar. Tudo quanto o Pai tem é meu; por isso é que vos disse que há de receber
do que é meu e vo-lo há de anunciar (Jo 16.12-15).
 
Vemos novamente que o ministério do Espírito Santo é aplicar a obra de
Cristo ao seu povo, e ele faz isto por santificar-nos, por revelar-nos a verdade
de Deus e por vir a nós em fortalecimento. O discurso de Jesus no cenáculo


In [8]:
from IPython.display import display, Markdown

def get_task_output(task):
    return f"""
**Tarefa:** \n
{task.output.description} \n \n
**Saída:** \n
{task.output.raw_output}
"""

Markdown(get_task_output(library_select_task))



**Tarefa:** 

Selecione os trechos que falam sobre o tópico: Julgamento do trono branco Explique a relação ou indique caso não haja relação. Responda em português. 
 

**Saída:** 

- Trecho de Somos Todos Teólogos - RC Sproul - página 210:
Neste trecho, o autor fala sobre a importância do Espírito da verdade, que dará testemunho de Cristo e guiará os discípulos à verdade. Isso está relacionado ao tópico do Julgamento do trono branco, pois menciona a vinda do Consolador que testemunhará de Cristo, o que pode ser associado ao momento de julgamento.
- Trecho de Somos Todos Teólogos - RC Sproul - página 213:
Neste trecho, é destacado o papel do Espírito Santo em guiar os discípulos à verdade e anunciar as coisas que hão de vir. Isso também está relacionado ao tópico do Julgamento do trono branco, pois mostra a importância da revelação divina e da obra de Cristo, que serão fundamentais nesse momento de julgamento.
- Trecho de Evangelho Maltrapilho - Brennan Manning - página 68:
Neste trecho, Jesus fala sobre a importância de permanecer Nele para encontrar um lugar seguro no meio da ansiedade do mundo. Embora não mencione diretamente o Julgamento do trono branco, a ideia de encontrar segurança em Cristo pode ser associada à esperança e proteção que os crentes terão nesse momento de julgamento.
- Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill - página 45:
Neste trecho, o autor questiona se os pregadores estão comunicando os "poderes do mundo vindouro" em seus cultos. Embora não faça menção direta ao Julgamento do trono branco, a reflexão sobre a intensidade espiritual e a batalha espiritual mencionada pode ser relacionada à preparação e importância desse momento de julgamento.


In [9]:
Markdown(result)

- Trecho de Somos Todos Teólogos - RC Sproul - página 210:
Neste trecho, o autor fala sobre a importância do Espírito da verdade, que dará testemunho de Cristo e guiará os discípulos à verdade. Isso está relacionado ao tópico do Julgamento do trono branco, pois menciona a vinda do Consolador que testemunhará de Cristo, o que pode ser associado ao momento de julgamento.
- Trecho de Somos Todos Teólogos - RC Sproul - página 213:
Neste trecho, é destacado o papel do Espírito Santo em guiar os discípulos à verdade e anunciar as coisas que hão de vir. Isso também está relacionado ao tópico do Julgamento do trono branco, pois mostra a importância da revelação divina e da obra de Cristo, que serão fundamentais nesse momento de julgamento.
- Trecho de Evangelho Maltrapilho - Brennan Manning - página 68:
Neste trecho, Jesus fala sobre a importância de permanecer Nele para encontrar um lugar seguro no meio da ansiedade do mundo. Embora não mencione diretamente o Julgamento do trono branco, a ideia de encontrar segurança em Cristo pode ser associada à esperança e proteção que os crentes terão nesse momento de julgamento.
- Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill - página 45:
Neste trecho, o autor questiona se os pregadores estão comunicando os "poderes do mundo vindouro" em seus cultos. Embora não faça menção direta ao Julgamento do trono branco, a reflexão sobre a intensidade espiritual e a batalha espiritual mencionada pode ser relacionada à preparação e importância desse momento de julgamento.

In [10]:
print(crew.usage_metrics)

{'total_tokens': 27156, 'prompt_tokens': 18576, 'completion_tokens': 8580, 'successful_requests': 18}


In [11]:
#todo: criar um script python para rodar e retornar tudo isso depois desenvolver o contexto e relações bíblicas contexto histórico etc